#### Hardware Setup:

- rp2040 board with micropython v1.20.0 ( other versions should work too )
- sensor : internal CPU temperature sensor - can be infuenced by placing your fingertip on the chip or by blowing on it
- connected via serial port

#### Software Setup:

This example uses bqplot to plot the data. bqplot is a plotting library for Jupyter notebooks. It is not part of MicroPython. You need to install it first.


In [1]:
# load / install libraries to display the plots
try:
    import bqplot.pyplot as plt
    import numpy as np
    print("bqplot and numpy loaded")
except (ModuleNotFoundError, ImportError):
    %pip install -U bqplot numpy


bqplot and numpy loaded


In [2]:
# start with an empty list of measurements (on the host computer)
measurements = []

#### Define a **micropython** function to read the temperature from the sensor.

The `%%micropython` magic is used to define a micropython function in a jupyter notebook cell.  
All code in the cell is passed to the micropython Microcontroller and executed on that (small) device.

Any output using print commands is returned to the notebook and displayed in the cell output.


In [3]:
# %%micropython
import machine


def cpu_temp():
    "Function to measure the internal temperature sensor of a rp2040 cpu in Celsius"
    sensor_temp = machine.ADC(4)
    conversion_factor = 3.3 / (65535)
    reading = sensor_temp.read_u16() * conversion_factor
    return 27 - (reading - 0.706) / 0.001721

run the function once on the MCU , get the result , and print it


In [4]:
t = %mpy --eval cpu_temp()
print (f"temperature: {t=} C")
print( type(t))

temperature: t=34.0666 C
<class 'float'>


So we can also add it to the list of measurements


In [5]:
# measure once and append to the list
t = %mpy --eval cpu_temp()
print(t)
measurements.append(t)
print(measurements)

34.0666
[34.0666]


Now lets start a graph of the temperature over time, and display that ( with the single mesurement )


In [6]:
# create a graph of the measurements using pyplot
cpu_graph = plt.plot(measurements)
cpu_graph.colors = ["red"]
cpu_graph.xlabel = "samples"
cpu_graph.ylabel = "CPU Temp"
cpu_graph.title = "CPU Temperature"
# cpu_graph.display_legend = True
plt.clear()  # clear any previous plots
plt.show()

Now lets add a few more measurements, and display them all on the same graph


In [7]:
import time

for i in range(100):
    t = %mpy --eval cpu_temp()
    measurements.append(t)
    plt.plot(measurements)
    time.sleep(0.1)